In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa, librosa.display

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
import scipy.io.wavfile
import spafe
from spafe.features.lfcc import lfcc
from spafe.features.mfcc import mfcc

In [4]:
all_data=[]
data_path_dict = {
    0: ["D:/Work/VIT/Capstone Project/Data/Total Denoised Data/False Denoised Augmented/" + file_path for file_path in os.listdir("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/False Denoised Augmented/")],
    1: ["D:/Work/VIT/Capstone Project/Data/Total Denoised Data/Marvin Denoised Augmented/" + file_path for file_path in os.listdir("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/Marvin Denoised Augmented/")]
}

In [5]:
for class_label, audio_files in data_path_dict.items():
    for one_file in audio_files:
        sig, sr = librosa.load(one_file)
        sig = sig / max(abs(sig))
        lfccs  = lfcc(sig, fs = sr, num_ceps=13)
        lfccs_processed = np.mean(lfccs.T, axis=0) # some pre-processing
        all_data.append([mfccs_processed, class_label])
    print(f"Info: Succesfully Preprocessed Class Label {class_label}")

df = pd.DataFrame(all_data, columns=["feature", "class_label"])
df.to_pickle("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/audio_data-lfcc-denoised.csv") #saving as csv for later use

Info: Succesfully Preprocessed Class Label 0
Info: Succesfully Preprocessed Class Label 1


In [6]:
df[df['class_label']==0].count()

feature        10614
class_label    10614
dtype: int64

In [7]:
df[df['class_label']==1].count()

feature        10476
class_label    10476
dtype: int64

In [8]:
df['feature'][0].shape

(98,)

In [9]:
p = 98
for i in range(len(df)):
  q = df['feature'][i].shape[0]
  while(q<p):
    df['feature'][i] = np.append(df['feature'][i],0)
    q+=1

C:\Users\91805\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [10]:
p = 98
cc = 0
for i in range(len(df)):
    q = df['feature'][i].shape[0]
    if(q!=p):
        print(i,q)
        cc+=1
print(cc)

0


In [11]:
##### Loading saved csv ##############
#df = pd.read_pickle("audio_data.csv")

####### Making our data training-ready
X = df["feature"].values
X = np.concatenate(X, axis=0).reshape(len(X), 98)

y = np.array(df["class_label"].tolist())
yy = y.copy()
y = to_categorical(y)

####### train test split ############
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=225)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.11, random_state=225)
print(X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val))

(16893, 98) (2109, 98) (2088, 98) 16893 2109 2088


In [12]:
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/X_train.npy",X_train)
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/y_train.npy",y_train)
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/X_test.npy",X_test)
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/y_test.npy", y_test)
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/X_val.npy", X_val)
np.save("D:/Work/VIT/Capstone Project/Data/Total Denoised Data/LFCCs Denoised/Augmented2/y_val.npy", y_val)